In [1]:
import json
import pandas as pd
import os
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', -1)

In [2]:
all_recipes = []
for file in os.listdir("./"):
    if ".json" in file:
        with open(file, "r") as scraped:
            loaded = json.load(scraped)
            for recipe in loaded:
                all_recipes.append(recipe)

In [3]:
with open("old_jsons.json", "w") as old_jsons:
    json.dump(all_recipes, old_jsons)

In [5]:
with open("old_jsons.json", "r") as old_jsons:
    loaded = json.load(old_jsons)
loaded

[{'name': 'Pancakes',
  'categories': ['Kuchnie świata', 'Kuchnia amerykańska'],
  'rating': 5,
  'raters': '1',
  'likes': '5',
  'duration': '90 minut',
  'ingredients': {'jajko': '1 szt.',
   'sól': '',
   'cukier': '1 łyżka',
   'mąka': '1 szklanka',
   'masło': '3 łyżki',
   'proszek do pieczenia': '0,5 łyżeczki',
   'soda oczyszczona': '0,5 łyżeczki',
   'maślanka': '1 szklanka',
   'jagody': '50 g'},
  'instructions': 'Krok 1\nDo mąki dodaj proszek do pieczenia, sodę, cukier oraz sól.\nKrok 2\nDo maślanki wlej 2 łyżki roztopionego masła oraz wbij jajko.\nKrok 3\nMąkę połącz z maślanką, wymieszaj, dodaj jagody i odstaw ciasto na godzinę.\nKrok 4\nPatelnię posmaruj pozostałą łyżką masła i smaż małe placki z obu stron na złoto.',
  'tags': ['kuchnia amerykańska',
   'jajka',
   'prosty przepis',
   'maślanka',
   'jagody'],
  'link': 'https://www.przepisy.pl/przepis/pancakes'},
 {'name': 'Krewetki zapiekane z czosnkiem',
  'categories': ['Dania i przekąski', 'Przystawki'],
  'ratin

In [6]:
len(loaded)

16497

In [3]:
r = pd.DataFrame(all_recipes)

In [4]:
r["likes"][r["likes"] == ""] = "0"
r["raters"][r["raters"] == ""] = "0"

c:\users\helen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\helen\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
print(r.dtypes)

categories      object 
duration        object 
ingredients     object 
instructions    object 
likes           float64
link            object 
name            object 
raters          float64
rating          float64
tags            object 
dtype: object


In [6]:
r.likes = r.likes.astype(float)
r.raters = r.raters.astype(float)

In [8]:
r["rating2"] = (r["raters"] * r["rating"] + (r["likes"] / 2) * 5) / (r["raters"] + (r["likes"]/ 2))

In [9]:
max_raters = (r["likes"] + (r["raters"] / 2)).max()

In [10]:
mean_rating = r["rating2"].mean()

In [11]:
r["rating3"] = ((r["likes"] + (r["raters"] / 2)) / max_raters) * mean_rating + (1 - ((r["likes"] + (r["raters"] / 2)) / max_raters)) * r["rating2"]

In [12]:
r["rating3"].max()

4.999996389828502

In [218]:
r.sort_values("rating3", ascending = False, inplace = True)

In [13]:
search_results = r[r["name"].str.contains("maszynkowe")]

In [14]:
search_results.ingredients.str.len().min()

5

In [221]:
search_results.index = search_results["ingredients"].str.len()
search_results = search_results.sort_index(ascending = True).reset_index(drop=True)
print(search_results.head(1))

  categories duration ingredients instructions  likes link name  raters  \
0  NaN        NaN      NaN         NaN         NaN     NaN  NaN NaN       

   rating tags  rating2  rating3  
0 NaN      NaN NaN      NaN       


In [222]:
r["ingr_len"] = r["ingredients"].apply(len)

In [223]:
r.sort_values("ingr_len", ascending = True, inplace = True) 

In [224]:
r = r[r["ingr_len"] > 1]

In [225]:
r = r.sort_values("rating3", ascending = False) 

In [226]:
r.head(20)

,categories,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3,ingr_len
3263,"[Dania i przekąski, Zupy]",60 minut,"{'suszone grzyby': '150 g', 'rosół z kury': '1 szt.'}","Krok 1\nGrzyby umyj, zalej przegotowaną wodą i pozostaw na noc.\nKrok 2\nNastępnego dnia ugotuj.\nKrok 3\nBulion wykonaj z kostki rosołowej Knorr, a następnie dodaj do niego wywar z grzybów.",0.0,https://www.przepisy.pl/przepis/bulion-z-suszonych-grzybow,Bulion z suszonych grzybów,0.0,0.0,"[zupy, prosty przepis, przepisy z 3 składników, frisco, pomysł na zupę]",NaN,NaN,2
2772,"[Dania i przekąski, Surówki]",15 minut,"{'seler naciowy': '1 pęczek', 'jogurt brzoskwiniowy duży': '1 opak.'}",Krok 1\nNać selera pokrój na drobne kawałki.\nKrok 2\nZalej jogurtem brzoskwiniowym i wymieszaj.,0.0,https://www.przepisy.pl/przepis/surowka-z-selera-naciowego-z-jogurtem-brzoskwiniowym,Surówka z selera naciowego z jogurtem brzoskwiniowym,0.0,0.0,"[prosty przepis, surówki, przepisy z 3 składników, szybkie danie]",NaN,NaN,2
9173,[],0 minut,"{'woda': '3 szklanki', 'kasza gryczana': '1 szklanka'}",Krok 1\nDo lekko osolonej wody wsyp szklankę kaszy gryczanej.\nKrok 2\nNastępnie gotuj kilka minut.\nKrok 3\nPrzykryj i gotuj na wolnym ogniu 30 minut.,0.0,https://www.przepisy.pl/przepis/kasza-gryczana-gotowana,Kasza gryczana gotowana,0.0,0.0,[],NaN,NaN,2
1557,"[Dania i przekąski, Przystawki]",90 minut,"{'mąka': '50 dag', 'cukier': '15 dag'}","Krok 1\nGrysik rozmieszać w niewielkiej ilości mleka. Resztę mleka zagotować, wlać grysik i zagotować. Ostudzić.\nKrok 2\nUtrzeć masło z cukrem do białości. Dodawać po trochu grysik i porządnie utrzeć. Na koniec dodać olejek rumowy.\nKrok 3\nPrzełożyć dwa placki powidłem śliwkowym, a następne kremem. Oprószyć cukrem pudrem.",0.0,https://www.przepisy.pl/przepis/placek-z-grysikiem,Placek z grysikiem,0.0,0.0,"[przystawki, prosty przepis, przepisy z 3 składników]",NaN,NaN,2
11618,"[Dania i przekąski, Dania mączne, Kluski, Kopytka]",45 minut,"{'ziemniak ugotowany': '1 kg', 'mąka pszenna': '3 szklanki'}","Krok 1\nWykonanie kopytek jest bardzo proste. Wystarczy dzień wcześniej ugotować więcej ziemniaków. I to wszystko.\nKrok 2\nZiemniaki przepuszczamy przez praskę, wyrabiamy z mąką. Formujemy wałeczki o średnicy ok 2 cm. Tniemy je na kopytka ok. 1-2 cm. Wrzucamy na osoloną wrzącą wodę i gotujemy do wypłynięcia.\nKrok 3\nKopytka podajemy z sosem mięsnym, smażoną cebulką lub skwarkami.",0.0,https://www.przepisy.pl/przepis/kopytka-3,Kopytka,0.0,3.0,"[kuchnia polska, prosty przepis, kopytka, przepisy z 3 składników]",NaN,NaN,2
11611,"[Dania i przekąski, Dania mączne, Kluski, Kopytka]",30 minut,"{'ziemniaki': '0,5 kg', 'mąka tortowa': '300 g'}","Krok 1\nUgotowane i wystudzone ziemniaki ugnieść. Włożyć do miski.\nKrok 2\nUgniatać ciasto dodając po trochę mąki , tak aby ciasto nie było ani za twarde, ani za luźne. Ciasto nie może się kleić.\nKrok 3\nPokroić kopytka, gotować na osolonym wrzątku do momentu wypłynięcia na powirzchnię.",0.0,https://www.przepisy.pl/przepis/delikatne-kopytka,Delikatne kopytka,0.0,3.0,"[kuchnia polska, prosty przepis, kopytka, przepisy z 3 składników, ziemniaki]",NaN,NaN,2
1558,"[Dania i przekąski, Przystawki]",15 minut,"{'parówka cienka': '6 szt.', 'makaron spagetti': '60 szt.'}",Krok 1\nParówki nadkłuwamy na wylot makaronem spagetti. Z obu stron taka sama długość powinna wystawać.\nKrok 2\nWrzucamy do gotującej osolonej wody. Gotujemy na małym ogniu aż makaron zrobi się miękki,0.0,https://www.przepisy.pl/przepis/stonoga,stonoga,0.0,0.0,"[mięso, drób, przystawki, makarony, prosty przepis, przepisy z 3 składników, szybkie danie]",NaN,NaN,2
1597,"[Dania i przekąski, Przystawki]",45 minut,"{'biała czekolada': '1 kg', 'mleko': '3 l'}","Krok 1\nWrzuć do garnka 1 tabliczkę białej czekolady, jak się rozpuści wsyp cukru, i mieszaj przez 3 minuty, następnie zostaw na 15 minut garnek by się nadal gotowało.\nKrok 2\nPóźniej nalej do garnka jedną drugą szklanki mleka i znowu mieszaj przez minutę. I po pomieszani

In [15]:
len(search_results)

2

In [16]:
search_results

,categories,duration,ingredients,instructions,likes,link,name,raters,rating,tags,rating2,rating3
6925,"[Ciasta, Ciasteczka, Ciasteczka kruche]",15 minut,"{'mąka': '2,5 szklanki', 'cukier puder': '0,5 szklanki', 'żółtko jajka': '5 szt.', 'czekolada': '0,5 opak.', 'kostka do pieczenia kasia': '1 opak.'}",Krok 1\nWszystkie składniki zagnieść na gładkie ciasto.\nKrok 2\nOdłożyć w chłodne miejsce na 30 minut.\nKrok 3\nPorcjami wkładać do maszynki i wykręcać paski długości około 10-12 cm.\nKrok 4\nUkładać na blasze wyłożonej papierem do pieczenia.\nKrok 5\nPo upieczeniu i ostygnięciu można polać polewą czekoladową.,21.0,https://www.przepisy.pl/przepis/kruche-ciasteczka-maszynkowe-7972,Kruche ciasteczka maszynkowe,9.0,4.5,"[czekolada, ciasteczka kruche, ciasteczka z czekoladą, bez mięsa, przepisy z 5 składników, frisco, szybkie danie, wege, danie wegetariańskie, ciasteczka, moje ciasto]",4.769231,4.769508
7326,"[Ciasta, Ciasteczka, Ciasteczka kruche]",60 minut,"{'żółtko gotowane': '7 szt.', 'surowe żółtko': '2 szt.', 'cukier puder': '0,5 szklanki', 'cukier waniliowy': '1 opak.', 'mąka': '0,5 kg', 'proszek do pieczenia': '0 łyżeczki', 'kostka do pieczenia kasia': '30 dag'}","Krok 1\nSurowe żółtka ucierać z cukrem pudrem i cukrem waniliowym dodając przetarte przez sitko gotowane żółtka-utrzeć do białości.\nKrok 2\nMąkę z proszkiem do pieczenia i Kasią posiekać nożem, dodając utarte żółtka, wyrabiając ciasto ręką.\nKrok 3\nCiastka formować przepuszczając przez maszynkę z odpowiednią nakładką.\nKrok 4\nPiec na złoty kolor.",2.0,https://www.przepisy.pl/przepis/ciastka-maszynkowe-5789,Ciastka maszynkowe,3.0,4.5,"[prosty przepis, ciasteczka kruche, bez mięsa, frisco, wege, danie wegetariańskie, ciasteczka]",4.625000,4.625078
